In [1]:
import altair
import pandas as pd
import numpy as np

In [4]:
df_sum = pd.read_excel('summary.xlsx')

In [5]:
df_sum

,Protease,Spectrums,Sites,Proteins
0,Trypsin,564,544,526
1,Chymotrypsin,353,339,323
2,Trypsin_GluC,597,557,524
3,Total,1514,1390,1256


In [27]:
altair.Chart(df_sum).transform_fold(fold=['Spectrums','Sites','Proteins'],as_=['Type','Count']).mark_bar().encode(x=altair.Row('Type:N',sort=['Spectrums','Sites','Proteins']),y='Count:Q',color='Type:N',column=altair.Row('Protease',sort=['Total','Trypsin','Chymotrypsin','Trypsin_GluC']))

alt.Chart(...)

In [26]:
altair.Chart(df_sum).transform_fold(['Spectrums','Sites','Proteins'],).mark_bar().encode(x=altair.Row('Protease',sort=['Total','Trypsin','Chymotrypsin','Trypsin_GluC']),y='value:Q',color='Protease:N',column=altair.Row('key:N',sort=['Spectrums','Sites','Proteins']))

alt.Chart(...)

In [7]:
def matrix_construct_adjust(seq, contain_mid=True, oneside_num=30):
    if contain_mid:
        if len(seq) != oneside_num * 2 + 3:
            raise Exception('seq len is wrong!')
    elif contain_mid == False:
        if len(seq) != oneside_num * 2:
            raise Exception('seq len is wrong!')

    if contain_mid:
        seq = seq[0:oneside_num] + seq[oneside_num + 3: oneside_num * 2 + 3]
    
    seq_matrix = np.zeros((oneside_num * 2,4))
    i = 0

    for nc in seq:
        if nc == 'A':
            nc = 0
        elif nc == 'T':
            nc = 1
        elif nc == 'C':
            nc = 2
        elif nc == 'G':
            nc = 3
        
        seq_matrix[i,nc] = 1
        i = i + 1
    
    return(seq_matrix)

def score_cal_adjust(seq_matrix, weights_matrix,oneside_num=30):
    weights_matrix = weights_matrix[30 - oneside_num : oneside_num + 30]
    result_matrix = weights_matrix * seq_matrix
    result = (np.sum(result_matrix) - np.sum(np.min(weights_matrix,axis=1))) / (np.sum(np.max(weights_matrix,axis=1)) - np.sum(np.min(weights_matrix,axis=1)))
    #result = np.sum(result_matrix) 
    return(result)

def matrix_construct_adjust_l(seq, contain_mid=True, oneside_num=30):
    seq_matrix = np.zeros((oneside_num,4))
    i = 0

    for nc in seq:
        if nc == 'A':
            nc = 0
        elif nc == 'T':
            nc = 1
        elif nc == 'C':
            nc = 2
        elif nc == 'G':
            nc = 3
        
        seq_matrix[i,nc] = 1
        i = i + 1
    
    return(seq_matrix)

def score_cal_adjust_l(seq_matrix, weights_matrix,oneside_num=30):
    weights_matrix = weights_matrix[30 - oneside_num : 30]
    result_matrix = weights_matrix * seq_matrix
    result = (np.sum(result_matrix) - np.sum(np.min(weights_matrix,axis=1))) / (np.sum(np.max(weights_matrix,axis=1)) - np.sum(np.min(weights_matrix,axis=1)))
    #result = np.sum(result_matrix) 
    return(result)

In [5]:
seq_eff = pd.read_excel('weights/test_data.xlsx',sheet_name='seq_eff')

In [62]:
f = open('weights/se_with_protein_bg_without_con_20230408.txt','r')
lines = f.readlines()

weights = np.zeros((60,4))

for line in lines:
    nc = line.split('[')[1].split('_')[0]
    if nc == 'A':
        nc = 0
    elif nc == 'T':
        nc = 1
    elif nc == 'C':
        nc = 2
    elif nc == 'G':
        nc = 3

    i = line.split('_')[1].split('=')[0]
    i = int(i)
    if i == 0:
        continue
    if i == 1:
        continue
    if i == -1:
        continue

    if i < 0:
        i = i + 31
    elif i > 0:
        i = i + 28
    
    w = line.split('=')[1].split(']')[0]
    w = float(w)

    weights[i,nc] = w

f.close()

In [63]:
row_sums = weights.sum(axis=1)
weights = weights / row_sums[:, np.newaxis]

n = 12
for index,row in seq_eff.iterrows():
    seq_eff.loc[index,'score'] = score_cal_adjust_l(matrix_construct_adjust_l(row.CutSeq63[30 - n : 30],oneside_num=n),weights,oneside_num=n)

altair.Chart(seq_eff).mark_point().encode(x='Ratio',y='score',color='Condition').properties(width=100,height=100).facet(facet='Condition',columns=4)

alt.FacetChart(...)

In [65]:
f = open('weights/weights_all.txt','r')
lines = f.readlines()

weights = np.zeros((60,4))

for line in lines:
    nc = line.split('[')[1].split('_')[0]
    if nc == 'A':
        nc = 0
    elif nc == 'T':
        nc = 1
    elif nc == 'C':
        nc = 2
    elif nc == 'G':
        nc = 3

    i = line.split('_')[1].split('=')[0]
    i = int(i)
    if i == 0:
        continue
    if i == 1:
        continue
    if i == -1:
        continue

    if i < 0:
        i = i + 31
    elif i > 0:
        i = i + 28
    
    w = line.split('=')[1].split(']')[0]
    w = float(w)

    weights[i,nc] = w

f.close()

In [66]:
row_sums = weights.sum(axis=1)
weights = weights / row_sums[:, np.newaxis]

In [68]:
n = 12
for index,row in seq_eff.iterrows():
    seq_eff.loc[index,'score'] = score_cal_adjust_l(matrix_construct_adjust_l(row.CutSeq63[30 - n : 30],oneside_num=n),weights,oneside_num=n)

altair.Chart(seq_eff).mark_point().encode(x='Ratio',y='score',color='Condition').properties(width=100,height=100).facet(facet='Condition',columns=4)

alt.FacetChart(...)

In [17]:
f = open('weights/weights_top100.txt','r')
lines = f.readlines()

weights = np.zeros((60,4))

for line in lines:
    nc = line.split('[')[1].split('_')[0]
    if nc == 'A':
        nc = 0
    elif nc == 'T':
        nc = 1
    elif nc == 'C':
        nc = 2
    elif nc == 'G':
        nc = 3

    i = line.split('_')[1].split('=')[0]
    i = int(i)
    if i == 0:
        continue
    if i == 1:
        continue
    if i == -1:
        continue

    if i < 0:
        i = i + 31
    elif i > 0:
        i = i + 28
    
    w = line.split('=')[1].split(']')[0]
    w = float(w)

    weights[i,nc] = w

f.close()

In [18]:
row_sums = weights.sum(axis=1)
weights = weights / row_sums[:, np.newaxis]

n = 12
for index,row in seq_eff.iterrows():
    seq_eff.loc[index,'score'] = score_cal_adjust_l(matrix_construct_adjust_l(row.CutSeq63[30 - n : 30],oneside_num=n),weights,oneside_num=n)

altair.Chart(seq_eff).mark_point().encode(x='Ratio',y='score',color='Condition').properties(width=100,height=100).facet(facet='Condition',columns=4)

alt.FacetChart(...)

In [62]:
f = open('weights/weights_top500.txt','r')
lines = f.readlines()

weights = np.zeros((60,4))

for line in lines:
    nc = line.split('[')[1].split('_')[0]
    if nc == 'A':
        nc = 0
    elif nc == 'T':
        nc = 1
    elif nc == 'C':
        nc = 2
    elif nc == 'G':
        nc = 3

    i = line.split('_')[1].split('=')[0]
    i = int(i)
    if i == 0:
        continue
    if i == 1:
        continue
    if i == -1:
        continue

    if i < 0:
        i = i + 31
    elif i > 0:
        i = i + 28
    
    w = line.split('=')[1].split(']')[0]
    w = float(w)

    weights[i,nc] = w

f.close()

In [63]:
row_sums = weights.sum(axis=1)
weights = weights / row_sums[:, np.newaxis]

In [64]:
n = 9
for index,row in seq_eff.iterrows():
    seq_eff.loc[index,'score'] = score_cal_adjust_l(matrix_construct_adjust_l(row.CutSeq63[30 - n : 30],oneside_num=n),weights,oneside_num=n)

altair.Chart(seq_eff).mark_point().encode(x='Ratio',y='score',color='Condition').properties(width=100,height=100).facet(facet='Condition',columns=4)

alt.FacetChart(...)

In [36]:
f = open('weights/weights_top50.txt','r')
lines = f.readlines()

weights = np.zeros((60,4))

for line in lines:
    nc = line.split('[')[1].split('_')[0]
    if nc == 'A':
        nc = 0
    elif nc == 'T':
        nc = 1
    elif nc == 'C':
        nc = 2
    elif nc == 'G':
        nc = 3

    i = line.split('_')[1].split('=')[0]
    i = int(i)
    if i == 0:
        continue
    if i == 1:
        continue
    if i == -1:
        continue

    if i < 0:
        i = i + 31
    elif i > 0:
        i = i + 28
    
    w = line.split('=')[1].split(']')[0]
    w = float(w)

    weights[i,nc] = w

f.close()

In [37]:
n = 12
for index,row in seq_eff.iterrows():
    seq_eff.loc[index,'score'] = score_cal_adjust_l(matrix_construct_adjust_l(row.CutSeq63[30 - n : 30],oneside_num=n),weights,oneside_num=n)

altair.Chart(seq_eff).mark_point().encode(x='Ratio',y='score',color='Condition').properties(width=100,height=100).facet(facet='Condition',columns=4)

alt.FacetChart(...)

In [38]:
f = open('weights/weights_top200.txt','r')
lines = f.readlines()

weights = np.zeros((60,4))

for line in lines:
    nc = line.split('[')[1].split('_')[0]
    if nc == 'A':
        nc = 0
    elif nc == 'T':
        nc = 1
    elif nc == 'C':
        nc = 2
    elif nc == 'G':
        nc = 3

    i = line.split('_')[1].split('=')[0]
    i = int(i)
    if i == 0:
        continue
    if i == 1:
        continue
    if i == -1:
        continue

    if i < 0:
        i = i + 31
    elif i > 0:
        i = i + 28
    
    w = line.split('=')[1].split(']')[0]
    w = float(w)

    weights[i,nc] = w

f.close()

In [39]:
n = 12
for index,row in seq_eff.iterrows():
    seq_eff.loc[index,'score'] = score_cal_adjust_l(matrix_construct_adjust_l(row.CutSeq63[30 - n : 30],oneside_num=n),weights,oneside_num=n)

altair.Chart(seq_eff).mark_point().encode(x='Ratio',y='score',color='Condition').properties(width=100,height=100).facet(facet='Condition',columns=4)

alt.FacetChart(...)

In [71]:
f = open('weights/weights_all_duplicate.txt','r')
lines = f.readlines()

weights = np.zeros((60,4))

for line in lines:
    nc = line.split('[')[1].split('_')[0]
    if nc == 'A':
        nc = 0
    elif nc == 'T':
        nc = 1
    elif nc == 'C':
        nc = 2
    elif nc == 'G':
        nc = 3

    i = line.split('_')[1].split('=')[0]
    i = int(i)
    if i == 0:
        continue
    if i == 1:
        continue
    if i == -1:
        continue

    if i < 0:
        i = i + 31
    elif i > 0:
        i = i + 28
    
    w = line.split('=')[1].split(']')[0]
    w = float(w)

    weights[i,nc] = w

f.close()

In [72]:
row_sums = weights.sum(axis=1)
weights = weights / row_sums[:, np.newaxis]

In [73]:
n = 12
for index,row in seq_eff.iterrows():
    seq_eff.loc[index,'score'] = score_cal_adjust_l(matrix_construct_adjust_l(row.CutSeq63[30 - n : 30],oneside_num=n),weights,oneside_num=n)

altair.Chart(seq_eff).mark_point().encode(x='Ratio',y='score',color='Condition').properties(width=100,height=100).facet(facet='Condition',columns=4)

alt.FacetChart(...)

In [86]:
weights = np.loadtxt('weights/kpLogo.txt')

In [84]:
row_sums = weights.max(axis=1) - weights.min(axis=1) 
weights = weights / row_sums[:, np.newaxis]

In [89]:
n = 15
for index,row in seq_eff.iterrows():
    seq_eff.loc[index,'score'] = score_cal_adjust_l(matrix_construct_adjust_l(row.CutSeq63[30 - n : 30],oneside_num=n),weights,oneside_num=n)

altair.Chart(seq_eff).mark_point().encode(x='Ratio',y='score',color='Condition').properties(width=100,height=100).facet(facet='Condition',columns=4)

alt.FacetChart(...)